In [ ]:
import os
os.chdir('..')

In [ ]:
import pandas as pd
import numpy as np
from tqdm import tqdm_notebook
import gc
from sklearn.metrics import roc_auc_score
from sklearn.metrics import f1_score
from sklearn.metrics import precision_recall_curve
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split

import keras
import keras.backend as K
from keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation
from keras.models import Model, Sequential
from keras import initializers, regularizers, optimizers, layers

import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

from scripts.utils import log_msg, precision_reall_f1_report
from scripts.models import shallow_NN, CustomStopper, keras_categorical

In [ ]:
args={}
args['data'] = 'data/sample_data_features.csv'
args['feature_space'] = 'data/feature_names.csv'
args['test_size'] = 0.2
args['seed'] = 123456

args['cv_results'] = 'results/cv_shallow_NN.csv'

In [ ]:
args['layer_nodes'] = [[512], [256], [128], [64], [32]]

args['NFOLDS'] = 3
args['num_classes'] = 2
args['batch_size'] =1000
args['max_epochs'] = 200
args['early_stop_start'] = 50
args['verbose'] = 0

In [ ]:
data = pd.read_csv(args['data'])
data.head()

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(data.drop(['sequence', 'label'], axis=1), 
                                                    data[['label']], 
                                                    test_size=args['test_size'], 
                                                    random_state=args['seed'])

### CV

In [ ]:
folds = KFold(n_splits=args['NFOLDS'], shuffle=True, random_state=args['seed'])

columns = X_train.columns


results = []
log_msg(">>> Start CV")
for i in range(len(args['layer_nodes'])):

    gc.collect()
    score = 0
    splits = folds.split(X_train, y_train)

    for fold_n, (train_index, valid_index) in enumerate(splits):

        K.clear_session()
        gc.collect()

        X_train_cv, X_valid = X_train[columns].iloc[train_index].values, X_train[columns].iloc[valid_index].values
        y_train_cv, y_valid = y_train.iloc[train_index].values, y_train.iloc[valid_index].values

        model = shallow_NN(len(columns), args['num_classes'], layer_nodes=args['layer_nodes'][i])

        early_stop = CustomStopper(monitor='val_loss', min_delta=0, patience=5, verbose=0, mode='min', 
                                   start_epoch=args['early_stop_start'],
                                   restore_best_weights=True)

        y_train_cv_categorical = keras_categorical(y_train_cv, args['num_classes'])
        y_valid_categorical = keras_categorical(y_valid, args['num_classes'])

        model.fit(X_train_cv, y_train_cv_categorical, 
                  batch_size=args['batch_size'], 
                  epochs=args['max_epochs'], 
                  validation_data=(X_valid, y_valid_categorical),
                  verbose=args['verbose'],
                  callbacks=[early_stop])

        y_pred_valid = model.predict(X_valid)

        score += roc_auc_score(y_valid, y_pred_valid[:, 1]) / args['NFOLDS']

        del X_train_cv, X_valid, y_train_cv, y_train_cv_categorical, y_valid, y_valid_categorical
        del model

    results.append({'layer_nodes' : args['layer_nodes'][i],
               'score': score})

    log_msg(f">>> layer_nodes={args['layer_nodes'][i]} Mean AUC = {score}")

log_msg(">>> Finished!")

results_df = pd.DataFrame(results)
results_df.head()


In [ ]:
results_df.to_csv(args['cv_results'], index=False)

### Final train and test

In [ ]:
args['final_params'] = {'layer_nodes': [512]}

In [ ]:
X_train_final, X_valid, y_train_final, y_valid = train_test_split(X_train, y_train, 
                                                                  test_size=0.1, random_state=args['seed'])
X_train_final = X_train_final.values
X_valid = X_valid.values
y_train_final = y_train_final.values
y_valid = y_valid.values

model = shallow_NN(len(columns), args['num_classes'], layer_nodes=args['final_params']['layer_nodes'])
    
early_stop = CustomStopper(monitor='val_loss', min_delta=0, patience=5, verbose=0, mode='min', 
                           start_epoch=args['early_stop_start'],
                           restore_best_weights=True)

y_train_final_categorical = keras_categorical(y_train_final, args['num_classes'])
y_valid_categorical = keras_categorical(y_valid, args['num_classes'])

log_msg(">>> Training ")
model.fit(X_train_final, y_train_final_categorical, 
          batch_size=args['batch_size'], 
          epochs=args['max_epochs'], 
          validation_data=(X_valid, y_valid_categorical),
          verbose=args['verbose'],
          callbacks=[early_stop])

log_msg(">>> Finished!")



In [ ]:
y_pred = model.predict(X_valid)
y_pred = y_pred[:,1]

precision, recall, thresholds = precision_recall_curve(y_valid, y_pred)

reports = precision_reall_f1_report(precision, recall, thresholds, 
                                    font_scale=2,
                                    linewidth=3,
                                    plot=False)

threshold = reports[reports['f1']==reports['f1'].max()]['threshold'].values[0]

print('Threshold to get the best F1 on validation set: ', threshold)

In [ ]:
y_pred = model.predict(X_test)
y_pred = y_pred[:,1]

print(f">>> AUC on Test set: {roc_auc_score(y_test, y_pred)}\n")

y_pred_label = [1 if i >= threshold else 0 for i in y_pred]

print(f">>> F1 on Test set (threshold {threshold}) : {f1_score(y_test, y_pred_label)}\n")


### Precision, Recall and F1 vs threshold on test set

In [ ]:
precision, recall, thresholds = precision_recall_curve(y_test, y_pred)

In [ ]:
reports = precision_reall_f1_report(precision, recall, thresholds, 
                                    font_scale=2,
                                    linewidth=3)

In [ ]:
reports.head()

In [ ]:
print('Best F1: ', reports['f1'].max())
print('Threshold:', reports[reports['f1']==reports['f1'].max()]['threshold'].values[0])